# Problem Set 7
## Daniel Tracht

## Question 1
In this question, we will be using the data from the strongdrink.txt file.

In [86]:
import pandas as pd

wine = pd.read_csv('data/strongdrink.txt')

wine.head()

,cultivar,alco,malic,ash,alk,magn,tot_phen,flav,nonfl_phen,proanth,color_int,hue,OD280rat,proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


### Part a
We wish to estimate a multinomial logistic regression model for the cultivar, where the baseline class is 3.  We will estimate on a training set containing 75 percent of our observations.

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.linear_model import LogisticRegression
import numpy as np

# Define the needed variables for this analysis
# Using .values so I don't have to define new matrix for subsquent problems
X = wine[["alco", "malic", "tot_phen", "color_int"]].values
y = wine["cultivar"].values

# Divide the data into 75% training and 25% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 20)

# Run the regression
LogReg = LogisticRegression(random_state=20, solver="lbfgs", multi_class='multinomial', max_iter=1000)
train = LogReg.fit(X_train, y_train)

# coefs and intercepts are ordered j=1, j=2, j=3
print("For j=1:")
print("Intercept:", train.intercept_[0])
print("Betas 1 through 4", train.coef_[0])

print("For j=2:")
print("Intercept:", train.intercept_[1])
print("Betas 1 through 4", train.coef_[1])

# generating classification_report
y_pred = LogReg.predict(X_test)
print("Our classification report:")
print(classification_report(y_test, y_pred))

# generating MSE
print('Test set MSE = ', mean_squared_error(y_test, y_pred))

For j=1:
Intercept: -24.02761652694668
Betas 1 through 4 [ 1.70173443 -0.26578756  1.22410094  0.02250699]
For j=2:
Intercept: 22.78073250150875
Betas 1 through 4 [-1.46629729 -0.33295144  0.66355615 -0.92268168]
Our classification report:
              precision    recall  f1-score   support

           1       0.87      1.00      0.93        13
           2       1.00      0.90      0.95        21
           3       1.00      1.00      1.00        10

   micro avg       0.95      0.95      0.95        44
   macro avg       0.96      0.97      0.96        44
weighted avg       0.96      0.95      0.96        44

Test set MSE =  0.045454545454545456


We find that the error rate for cultivar 1 is the only one that is not zero for the quarter of the data we left as a test.  Using the f1-score as our measure, our model was best at classifying wines of cultivar 3.  This is also the category with the fewest observations (and so fewest chances to make a mistake).

### Part b
In this question, we want to use LOOCV with the model from part a.  We want to report our error rates for each category.  We want to report the LOOCV MSE for each test, as well as the average across all tests:

In [88]:
from sklearn.model_selection import LeaveOneOut

# Setting up with the number of observations, a vector of 0s to hold MSEs, a matrix to hold precision
N_loo = X.shape[0]
loo = LeaveOneOut()
loo.get_n_splits(X)
MSE_loo = np.zeros(N_loo)

# Adding a column of zeros to our data
wine["zeros"] = 0
# making matrix with our y and the column of zeros
precision_loo = wine[["cultivar", "zeros"]].values

In [89]:
# Running the loop
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression(random_state=20, solver="lbfgs", multi_class='multinomial', max_iter=1000)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    # have to use indicator since multinomial
    if y_pred == y_test:
        MSE_loo[test_index] = 0
        precision_loo[test_index,1] = 1
    else:
        MSE_loo[test_index] = 1

In [90]:
# Defining the subsets of the larger precision matrix for the three cultivars
prec_1 = precision_loo[np.where(precision_matrix[:,0] == 1)]
prec_2 = precision_loo[np.where(precision_matrix[:,0] == 2)]
prec_3 = precision_loo[np.where(precision_matrix[:,0] == 3)]

print("Error rate for cultivar 1:", 1 - prec_1[:,1].mean())
print("Error rate for cultivar 2:", 1 - prec_2[:,1].mean())
print("Error rate for cultivar 3:", 1 - prec_3[:,1].mean())

print("The MSE for each of the LOOCV tests: \n", MSE_loo)
print("Mean MSE across all LOOCV tests", MSE_loo.mean())

Error rate for cultivar 1: 0.06779661016949157
Error rate for cultivar 2: 0.09859154929577463
Error rate for cultivar 3: 0.06521739130434778
The MSE for each of the LOOCV tests: 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
Mean MSE across all LOOCV tests 0.07954545454545454


While the error rate for cultivar 1 decreased from 0.13 to 0.07, the error rates from cultivars 2 and 3 increased from 0 to 0.10 and 0.07 respectively.

### Part c
In this question, we want to use $k$-fold cross validation in which $k=4$.

In [98]:
from sklearn.model_selection import KFold

# Using the code as specified
kf = KFold(n_splits=4, shuffle=True, random_state=10)
kf.get_n_splits(X)

MSE_kf = np.zeros(4)
N_obs = X.shape[0]
precision_kf = np.zeros(10,2)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression(random_state=10, solver="lbfgs", multi_class='multinomial', max_iter=1000)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    print(y_test==y_pred)
    MSE_kf[k_ind] = ((y_test == y_pred) ** 2).mean()
    print('MSE for test set', k_ind, ' is', MSE_kf[k_ind])
    k_ind += 1

MSE_kf_mean = MSE_kf.mean()
print('test estimate MSE k-fold=', MSE_kf)

TypeError: data type not understood

In [ ]:
    # have to use indicator since multinomial
    if y_pred == y_test:
        MSE_kf[test_index] = 0
        #precision_matrix[test_index,1] = 1
    else:
        MSE_loo[test_index] = 1    

# Resampling Methods
### by [Richard W. Evans](https://sites.google.com/site/rickecon/), February 2019
The code in this Jupyter notebook was written using Python 3.6. It uses data files [`Titanic dataset`](https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv). For the code to run properly, you will either need to have access to the internet or you should have the data file in the same folder as the Jupyter notebook file. Otherwise, you will have to change the respective lines of the code that read in the data to reflect the location of that data. Some other good resampling content in R is Dr. Benjamin Soltoff's resampling methods notes [here](http://cfss.uchicago.edu/persp006_resampling.html).

Resampling methods are a way to test the sensitivity of statistical results to estimation using a different sample. It is often too difficult or too expensive to draw a new sample from the population. Resampling methods take advantage of the training-set test-set paradigm to evaluate the sensitivity of estimates to sample variance. The two main classes of resampling methods are:

1. Cross validation
2. Bootstrapping

In choosing models to predict or match data or to infer relationships between variables, James, et al (2013) decompose the process into *model assessment* and *model selection*. Model assessment is treated in this notebook. It is the process and various means of evaluating the fit or accuracy of a given model. Model selection is the process of adjusting parameters, variables, or functional relationships between variables to better fit the data.

## 1. Cross validation

### 1.1. Validation set approach
This is an approach that we have already studied in the [classifiers notebook](https://github.com/UC-MACSS/persp-model-econ_W19/blob/master/Notebooks/Classification/LogitKNN.ipynb).

1. Partition the data into a training set and a test set.
2. Estimate the model using the training set.
3. Evaluate the fit or predictive accuracy on the test set.

The primary measure of fit is the mean squared error (MSE) of the estimated model on the test set. Let the test set have $N$ observations. The MSE of the test set is the sum of squared deviations of the actual dependent variable values minus the predicted values.

$$ MSE = \frac{1}{N}\sum_{i=1}^N\left(y_i - \hat{y}_i\right)^2 $$

In classification problems, researchers sometimes use the ROC curve (receiver operating characteristics) and the area under the ROC curve (AUROC) because it captures both type 1 and type 2 errors in a single visualization (false positive rate versus true positive rate). You want the false positive rate to be low for any given true positive rate. The area under the ROC curve (AUROC). The AUROC is the opposite of a measure of error. The bigger the AUROC, the more accurate predictor is the model.

[Insert figure]

Let's calculate the MSE from our logistic regression of the titanic example.

In [ ]:
# Import needed stuff
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn import metrics 
from sklearn.metrics import classification_report, mean_squared_error
from pylab import rcParams

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
rcParams['figure.figsize'] = 10, 8
sb.set_style('whitegrid')

In [ ]:
# Read in Titanic data
url = ('https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/' +
      'master/titanic-train.csv')
titanic = pd.read_csv(url)
titanic.columns = ['PassengerId','Survived','Pclass','Name','Sex','Age',
                   'SibSp','Parch','Ticket','Fare','Cabin','Embarked']

# Get rid of columns we don't use
titanic = titanic.drop(['PassengerId','Name','Ticket','Cabin'], 1)

# Impute missing age values
def age_approx(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age

titanic['Age'] = \
    titanic[['Age', 'Pclass']].apply(age_approx, axis=1)
    
# Drop any observations with missing values
titanic.dropna(inplace=True)

# Make gender dummies and embark dummies and get rid of
# original variables
gender = pd.get_dummies(titanic['Sex'], drop_first=True)
embark_location = pd.get_dummies(titanic['Embarked'],
                                 drop_first=True)
titanic.drop(['Sex', 'Embarked'], axis=1, inplace=True)
titanic = pd.concat([titanic, gender, embark_location], axis=1)

# Drop Pclass variable due to excessive correlation with Fare
titanic.drop(['Pclass'], axis=1, inplace=True)

titanic.head()

Now partition the data into the same 60% training set sample that we did in the [logistic regression notebook](https://github.com/UC-MACSS/persp-model_W18/blob/master/Notebooks/Classfcn1/KKNlogitLDA.ipynb) and estimate the logistic regression with all the variables.

In [ ]:
X = titanic[['Age', 'SibSp', 'Parch', 'Fare', 'male', 'Q', 'S']]
y = titanic['Survived']
# This function train_test_split is from sklearn.cross_validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                    random_state=25)
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)
y_pred = LogReg.predict(X_test)
# Note that the squared doesn't matter in a Logistic model

# You can code the MSE yourself
MSE_vs = ((y_test - y_pred) ** 2).sum() / y_pred.shape[0]
print('Validation set MSE = ', MSE_vs)

# Or you can use scikit-learn's method
print('Validation set MSE = ', mean_squared_error(y_test, y_pred))

### 1.2. Leave-one-out cross validation
Leave-one-out cross validation (LOOCV) is an approach in which the model is assessed using $N$ different training sets and test sets of a specific size. Let the data have $N$ observations. LOOCV is to choose a training set with $N-1$ observations, such that the test set only has one observation $y_i$. Repeat this $N$ with a slightly different training set such that each data point is the test set in exactly one of these sebsets.

In this case, the mean squared error MSE has no summation because there is only one observation in the test set.

$$ MSE_i = (y_i - \hat{y}_i)^2 $$

The LOOCV estimate for the test MSE is the average of these $N$ test error estimates.

$$ CV_{loo} = \frac{1}{N}\sum_{i=1}^N MSE_i $$

In [ ]:
# Define loo as a leave-one-out object, then
# split it into N different partitions

# Note that the LeaveOneOut() function does not work
# well with pandas DataFrames
Xvars = titanic[['Age', 'SibSp', 'Parch', 'Fare',
                 'male', 'Q', 'S']].values
yvars = titanic['Survived'].values
N_loo = Xvars.shape[0]
loo = LeaveOneOut()
loo.get_n_splits(Xvars)
MSE_vec = np.zeros(N_loo)

# This loop will take 20 or 30 seconds
for train_index, test_index in loo.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec[test_index] = (y_test - y_pred) ** 2
    print('MSE for test set', test_index, ' is', MSE_vec[test_index])

MSE_loo = MSE_vec.mean()
MSE_loo_std = MSE_vec.std()
print('test estimate MSE loocv=', MSE_loo,
      ', test estimate MSE standard err=', MSE_loo_std)

### 1.3. k-fold cross validation
$k$-fold cross validation is a method in which the dataset is randomly divided into $k$ groups (folds). Define a test set of the model as the $k$th fold. For each test set $k$, the model is estimated on the data from the other $k-1$ folds. Let the number of observations in the $k$th fold be $N_k$, and let $\mathcal{K}$ be the set of observations in the $k$th fold. The $MSE_k$ of the $k$th fold is:

$$ MSE_k = \frac{1}{N_k}\sum_{i\in\mathcal{K}}(y_i - \hat{y}_i)^2 $$

Then the $k$-fold estimate for the test MSE is the average of these $k$ test error estimates.

$$ CV_{kf} = \frac{1}{k}\sum_{j=1}^k MSE_j $$

LOOCV is a special case of $k$-fold cross validation in which $k=N$.

Let's use the Titanic data again and test our logit model performance with a $k$-fold cross validation with $k=6$.

In [ ]:
k = 2
kf = KFold(n_splits=k, random_state=10, shuffle=True)
kf.get_n_splits(Xvars)

MSE_vec_kf = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('k index=', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('test estimate MSE k-fold=', MSE_kf,
      'test estimate MSE standard err=', MSE_kf_std)

### 1.4. Bias versus variance
Recall the test estimate MSE from the LOOCV of approximately 0.2115 and the MSE(LOOCV) standard error of about 0.4084. What happens to the estimated MSE and MSE standard error in the $k$-fold cross validation above as $k$ increases? Try values of $k=2, 10, 50, 100, 800$.

Note that the LOOCV method has low bias (estimated on large number of data) but high variance (errors are based on one draw). In contrast, the $k$-fold method has more bias (estimated with less data) but lower variance. Each test set has more observations.

* $k$-fold cross validation can often provide more accurate estimates of the test error rate.
* $k$-fold is less computationally intensive
* LOOCV has the least bias
* LOOCV is the most computationally expensive

## 2. Bootstrapping
This name comes from the expression "to pull oneself up by ones own bootstraps." In a way similar to the cross validation methods of the last section, we can use *the bootstrap* to quantify the undertainty associated with a given estimator, learning model, or method. In the econometrics and statistics literature, this often shows up as "bootstrapped standard errors". Bootstrapping is valuable because it is so widely applicable to a range of models.

1. Randomly draw $S$ datasets of size $N_S$ with replacement. Define each training set of observations as $\mathcal{K}_s$ and each corresponding test set as $\mathcal{-K}_{s}$.
2. Calculate the MSE for each test set $\mathcal{-K}_{s}$

The bootstrap estimate for the test MSE is the average MSE from each random test set.

$$ CV_{boot} = \frac{1}{S}\sum_{s=1}^S MSE_s $$

In [ ]:
N_bs = 100

MSE_vec_bs = np.zeros(N_bs)

for bs_ind in range(N_bs):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.4)
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_bs[bs_ind] = ((y_test - y_pred) ** 2).mean()
    print('MSE for test set', bs_ind, ' is', MSE_vec_bs[bs_ind])

MSE_bs = MSE_vec_bs.mean()
MSE_bs_std = MSE_vec_bs.std()
print('test estimate MSE bootstrap=', MSE_bs,
      'test estimate MSE standard err=', MSE_bs_std)

## 3. How to use cross validation for model assessment and selection
*Model assessment* is the process of evaluating the performance of a particular model estimated on training data on its prediction accuracy on test data. There are many criteria for model assessment. The most common measure of model accuracy on test data is the mean squared error $MSE$ or root mean squared error $rMSE$. However, we have seen that the measure $MSE$ varies depending on which cross validation method is used.

[JWHT13] define *Model selection* as the process of "selecting the proper level of flexibility for a model. That is, they define model selection as a process of tuning a particular family or class of models to maximize accuracy on test set prediction. So model selection involves model assessment. However, one can expand this definition of model selection to include testing multiple families or classes of model in terms of accuracy--a horse race.

The narrower [JWHT13] definition of model selection is analogous to maximizing the efficiency of a particular horse and rider in a horse race in which the horse is racing against itself. My broader definition of model selection is analogous running a number of horses in a race, with each horse being optimized for efficiency. In this broader definition, many variables are at play and the data must be sampled many times (think of cross-validation optimization on each horse [model]).

This broader definition of model selection is computationally intensive. But that is where TensorFlow shines. [TensorFlow](https://www.tensorflow.org/) is an open source software library developed by the Google Brain AI group. Broadly, TensorFlow is a system of libraries that facilitate efficient, parallel, and scalable use of available processors (CPUs and GPUs) as well as memory management. For statistical learning, TensorFlow is optimized to efficiently run model assessment and model selection algorithms.

Many good empirical papers run a horse race on tuned statistical learning models to maximize predictive accuracy. It is hard to know *ex ante* which model will be the most accurate.

### 3.1. Gopalan, "Predicting Infant Mortality: Minimizing False Negatives"
[G18] is able to maintain overall accuracy, and decrease false negatives from 74% to 7%. She uses a regularization method on one of her variables (class), called "Tomek links", to make the variable more informative. She then tests five different predictive models (random forest, AdaBost, XGBoost, decision tree, logistic regression). She also tried a couple of additional data transformations. In her case, the random forest model had the best performance.

**[G13, Table 6] Comparison of Classifiers: Tomek Links.** FNR=false negative rate, FPR=False positive rate, AUROC=area under the ROC curve.

| Model | Train-FNR | Train-FPR  | Train-AUROC | Test-FNR  | Test-FPR  | Test-AUROC |
| --- | --- | --- | --- | --- | --- | --- |
| Random Forest | 0.06 | 0.06 | 0.62  | 0.07 | 0.06 | 0.62  |
| AdaBoost      | 0.40 | 0.07 | 0.51  | 0.45 | 0.07 | 0.50  |
| XGBoost       | 0.40 | 0.07 | 0.51  | 0.43 | 0.07 | 0.51  |
| Decision Tree | 0.03 | 0.06 | 0.62  | 0.07 | 0.06 | 0.62  |
| Logistic Reg  | 0.41 | 0.07 | 0.51  | 0.43 | 0.07 | 0.65  |

## 4. Problem with full-sample estimation
Overfitting. Maximizing accuracy in training set can pick incorporate noise from the data.

## 5. References
* Gopalan, Sushmita V., "Predicting Infant Mortality: Minimizing False Negatives," MACSS Thesis, University of Chicago (April 2018).
* James, Gareth, Deaniela Witten, Trevor Hastie, and Robert Tibshirani, [*An Introduction to Statistical Learning with Applications in R*](http://link.springer.com.proxy.uchicago.edu/book/10.1007%2F978-1-4614-7138-7), New York, Springer (2013).